In [67]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from surprise import Dataset, Reader,accuracy, SVD
from surprise.model_selection import train_test_split

In [23]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
tags = pd.read_csv('tags.csv')

In [257]:
mov_rat = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
mov_rat.dropna(inplace=True)

mov_rat.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [258]:
#выбираем произвольного пользователя и определяем последний просмотренный им фильм с максимальной оценкой.
user = 10
user_data = mov_rat.loc[mov_rat['userId'] == user]
user_data = user_data.sort_values(by=['rating', 'timestamp'], ascending = False)
user_data.head()

,movieId,title,genres,userId,rating,timestamp
84016,49286,"Holiday, The (2006)",Comedy|Romance,10.0,5.0,1.455358e+09
99277,140110,The Intern (2015),Comedy,10.0,5.0,1.455357e+09
75201,7458,Troy (2004),Action|Adventure|Drama|War,10.0,5.0,1.455357e+09
91335,79091,Despicable Me (2010),Animation|Children|Comedy|Crime,10.0,5.0,1.455306e+09
77158,8869,First Daughter (2004),Comedy|Romance,10.0,5.0,1.455303e+09


In [205]:
user_data['genres'] = user_data['genres'].str.replace('Sci-Fi','SciFi')
user_data['genres'] = user_data['genres'].str.replace('Film-Noir','Noir')
user_data['genres'] = user_data['genres'].apply(lambda x: x.replace('|', ' ')).str.lower()

In [207]:
#дальше найдем 50 максимально похожих фильмов, используя жанры и модель поиска ближайшего соседа

In [274]:
tfidf_vec = TfidfVectorizer()
X_tfidf = tfidf_vec.fit_transform(movies['genres'])

from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors = 50)
neigh.fit(X_tfidf)

X1_tfidf = tfidf_vec.transform([user_data.iloc[0]['genres']])


In [269]:
# соберем список похожих фильмов
res = neigh.kneighbors(X1_tfidf, return_distance=False)

movie_for_rec =[]

for i in res[0]:
    film = movies.iloc[i]
    movie_for_rec.append(film.title)

50

In [217]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
movielens = fetch_movielens()

In [272]:
# используя light fm найдем рекомендации по всем фильмам для конкретного пользователя
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(movielens['train'], epochs=10)

n_users, n_items = movielens['train'].shape
scores = model.predict(user, np.arange(n_items))
top_items = movielens['item_labels'][np.argsort(-scores)]
top_items = top_items[:500]


In [271]:
#Окончательную рекомендацию формируем на пересечении двух полученных списков - ближайших соседей по жанру и общие рекомендации по скрытым факторам

for film in movie_for_rec:
    if film in top_items: print(film)

My Best Friend's Wedding (1997)
Dave (1993)
